## Exporting scRNAseq data using Python

In [1]:
import scanpy as sc
import pandas as pd

# read h5ad files.
adata_ref = sc.read_h5ad("/disk2/user/hilpek/project_root/scRNAseq/input/Wu_all_NT.h5ad")

# Removed HER samples.
adata_ref = adata_ref[(adata_ref.obs['IHC_subtype'] == 'ER+'), :]

# mt and ribo columns have been added to adata.var
adata_ref.var['mt'] = adata_ref.var_names.str.upper().str.startswith('MT-')
adata_ref.var['ribo'] = adata_ref.var_names.str.upper().str.startswith(('RPL', 'RPS'))
adata_ref.var_names_make_unique()

# Get ham counts matrix as a dataframe with gene and cell types.
import numpy as np
counts = pd.DataFrame(
    adata_ref.X.toarray() if hasattr(adata_ref.X, "toarray") else np.array(adata_ref.X),
    index=adata_ref.obs_names,
    columns=adata_ref.var_names
)
counts.T.to_csv("/disk2/user/hilpek/project_root/scRNAseq/input2/dge.csv")

/tmp/ipykernel_376608/3713809626.py:11: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata_ref.var['mt'] = adata_ref.var_names.str.upper().str.startswith('MT-')


In [2]:
adata_ref

AnnData object with n_obs × n_vars = 33790 × 29733
    obs: 'Study_id', 'Celltype_subset', 'bc_subtype', 'donor_age', 'treatment_status', 'grade', 'IHC_subtype', 'Dataset'
    var: 'ENSEMBL', 'mt', 'ribo'

In [3]:
adata_ref.obs

,Study_id,Celltype_subset,bc_subtype,donor_age,treatment_status,grade,IHC_subtype,Dataset
CID3941_AAAGTAGTCCTTGCCA,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941
CID3941_ACAGCCGAGCTCTCGG,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941
CID3941_ACGCAGCAGGACCACA,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941
CID3941_ACGCAGCCAGCTGTAT,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941
CID3941_ACTTTCACAGCCTGTG,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941
...,...,...,...,...,...,...,...,...
CID4535_TAGTGGTCATTGGCGC,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535
CID4535_TCTATTGAGCTAGGCA,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535
CID4535_TGCTACCAGGTGTGGT,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535
CID4535_TGGCCAGAGCCCAACC,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535


#### Get major cell types

In [4]:
# load metadata
metadata = pd.read_csv("/disk2/user/hilpek/project_root/scRNAseq/input/metadata.csv", sep=",")
metadata.set_index('Unnamed: 0', inplace=True)
metadata

# get corresponding major cell types for each subset of cell type
subset_to_major_dict = {}
for index, row in metadata.iterrows():
    subset = row["celltype_subset"]
    major = row["celltype_major"]
    if subset not in subset_to_major_dict.keys():
        subset_to_major_dict[subset] = major

subset_to_major_dict

{'Endothelial ACKR1': 'Endothelial',
 'Endothelial RGS5': 'Endothelial',
 'Endothelial CXCL12': 'Endothelial',
 'CAFs MSC iCAF-like s1': 'CAFs',
 'CAFs MSC iCAF-like s2': 'CAFs',
 'CAFs Transitioning s3': 'CAFs',
 'CAFs myCAF like s4': 'CAFs',
 'CAFs myCAF like s5': 'CAFs',
 'PVL Differentiated s3': 'PVL',
 'PVL_Immature s2': 'PVL',
 'PVL Immature s1': 'PVL',
 'Endothelial Lymphatic LYVE1': 'Endothelial',
 'B cells Memory': 'B-cells',
 'B cells Naive': 'B-cells',
 'T_cells_c4_CD8+_ZFP36': 'T-cells',
 'T_cells_c6_IFIT1': 'T-cells',
 'T_cells_c5_CD8+_GZMK': 'T-cells',
 'T_cells_c7_CD8+_IFNG': 'T-cells',
 'T_cells_c8_CD8+_LAG3': 'T-cells',
 'T_cells_c0_CD4+_CCR7': 'T-cells',
 'T_cells_c1_CD4+_IL7R': 'T-cells',
 'T_cells_c2_CD4+_T-regs_FOXP3': 'T-cells',
 'T_cells_c3_CD4+_Tfh_CXCL13': 'T-cells',
 'T_cells_c9_NK_cells_AREG': 'T-cells',
 'T_cells_c11_MKI67': 'T-cells',
 'T_cells_c10_NKT_cells_FCGR3A': 'T-cells',
 'Myeloid_c10_Macrophage_1_EGR1': 'Myeloid',
 'Myeloid_c12_Monocyte_1_IL1B': 'My

In [5]:
# save into adata.obs
adata_ref.obs['Celltype_major'] = adata_ref.obs['Celltype_subset'].map(subset_to_major_dict)
adata_ref.obs

,Study_id,Celltype_subset,bc_subtype,donor_age,treatment_status,grade,IHC_subtype,Dataset,Celltype_major
CID3941_AAAGTAGTCCTTGCCA,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941,Endothelial
CID3941_ACAGCCGAGCTCTCGG,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941,Endothelial
CID3941_ACGCAGCAGGACCACA,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941,Endothelial
CID3941_ACGCAGCCAGCTGTAT,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941,Endothelial
CID3941_ACTTTCACAGCCTGTG,CID3941,Endothelial ACKR1,IDC,50.0,Naive,2.0,ER+,CID3941,Endothelial
...,...,...,...,...,...,...,...,...,...
CID4535_TAGTGGTCATTGGCGC,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535,Normal Epithelial
CID4535_TCTATTGAGCTAGGCA,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535,Normal Epithelial
CID4535_TGCTACCAGGTGTGGT,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535,Normal Epithelial
CID4535_TGGCCAGAGCCCAACC,CID4535,Mature Luminal,ILC,47.0,Naive,2.0,ER+,CID4535,Normal Epithelial


In [6]:
adata_ref

AnnData object with n_obs × n_vars = 33790 × 29733
    obs: 'Study_id', 'Celltype_subset', 'bc_subtype', 'donor_age', 'treatment_status', 'grade', 'IHC_subtype', 'Dataset', 'Celltype_major'
    var: 'ENSEMBL', 'mt', 'ribo'

In [7]:
import numpy as np
import pandas as pd

# calculate nUMI
nUMI = adata_ref.X.sum(axis=1)
if hasattr(nUMI, "A1"):
    nUMI = nUMI.A1
else:
    nUMI = np.array(nUMI).flatten()

# create metadata
meta = pd.DataFrame({
    "barcode": adata_ref.obs_names.values,                      # cell barcodes
    "cluster": adata_ref.obs["Celltype_subset"].values,         # cell types
    "nUMI": nUMI                                                # total UMI
})

# save file
meta.to_csv("/disk2/user/hilpek/project_root/scRNAseq/input2/meta_data.csv", index=False)


## Import the raw count matrix and spatial coordinates into R

In [8]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import squidpy as sq
from scipy.sparse import csr_matrix
import anndata as ad
import os
import seaborn as sns
from sklearn.metrics import confusion_matrix

/disk2/user/hilpek/miniconda3/envs/cell2_env/lib/python3.9/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/disk2/user/hilpek/miniconda3/envs/cell2_env/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [9]:
import squidpy as sq
adata_vis = sq.read.visium("/disk3/cda/SpatialTranscriptomics/processed-data/SpaceRanger-output_v1.0.0-rerun2023/V10F03-034_C/outs",
        counts_file='filtered_feature_bc_matrix.h5', 
        source_image_path="/disk3/cda/SpatialTranscriptomics/raw-data/High-resolution_tissue_images/V10F03-034/210223_BC_S7_V10F03-034_RJ.C1-Spot000001.jpg"
    )

/disk2/user/hilpek/miniconda3/envs/cell2_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/disk2/user/hilpek/miniconda3/envs/cell2_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [10]:
# get the sample id for each spot
adata_vis.obs['sample'] = list(adata_vis.uns['spatial'].keys())[0]

In [11]:
# add pathologist annotations
annoDf = pd.read_csv("/disk3/cda/SpatialTranscriptomics/pathologist_annotations/Tissue-annotations_pathologist-2-and-3_CSVs/V10F03-034_C_S7_Wenwen-annotations.csv")

# replace empty annotations with "Mixed"
annoDf.fillna('Mixed', inplace=True)
annoArray = annoDf["Wenwen annotations"].to_numpy()
adata_vis.obs["Pathologist_annotations"] = pd.Categorical(annoArray)

In [12]:
# add sample name to obs names, important for looking at individual slides later
adata_vis.obs["sample"] = [str(i) for i in adata_vis.obs['sample']]
adata_vis.obs_names = adata_vis.obs["sample"] + '_' + adata_vis.obs_names
adata_vis.obs.index.name = 'spot_id'

In [13]:
adata_vis.var['SYMBOL'] = adata_vis.var_names
adata_vis.var.set_index('gene_ids', drop=True, inplace=True)

In [14]:
# calculate QC metrics
if not isinstance(adata_vis.X, np.ndarray):
    adata_vis.X = adata_vis.X.toarray()
    
adata_vis.var['mt'] = adata_vis.var["SYMBOL"].str.startswith(("mt-", "MT-"))
adata_vis.var["ribo"] = adata_vis.var["SYMBOL"].str.startswith(("RPS", "RPL"))

sc.pp.calculate_qc_metrics(adata_vis, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)

adata_vis.X = csr_matrix(adata_vis.X)

In [15]:
# find and remove MT genes for spatial mapping (keeping their counts in the object)
adata_vis.obs['mt_frac'] = adata_vis[:, adata_vis.var['mt'].tolist()].X.sum(1).A.squeeze()/adata_vis.obs['total_counts']
adata_vis.obsm['MT'] = adata_vis[:, adata_vis.var['mt'].values].X.toarray()

In [16]:
adata_vis = adata_vis[:, ~adata_vis.var['mt'].values]

In [17]:
# find and remove RB genes for spatial mapping (keeping their counts in the objm)
adata_vis.obs['ribo_frac'] = adata_vis[:, adata_vis.var['ribo'].tolist()].X.sum(1).A.squeeze()/adata_vis.obs['total_counts']
adata_vis.obsm['RB'] = adata_vis[:, adata_vis.var['ribo'].values].X.toarray()

/tmp/ipykernel_376608/2966163283.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_vis.obs['ribo_frac'] = adata_vis[:, adata_vis.var['ribo'].tolist()].X.sum(1).A.squeeze()/adata_vis.obs['total_counts']


In [18]:
adata_vis = adata_vis[:, ~adata_vis.var['ribo'].values]

In [19]:
 # choose appropriate thresholds for ST data
sc.pp.filter_cells(adata_vis, min_genes = 200)
sc.pp.filter_cells(adata_vis, max_counts= 3000)
adata_vis = adata_vis[adata_vis.obs["pct_counts_mt"] < 5]

# filter genes that are only present in a few spots
sc.pp.filter_genes(adata_vis, min_cells=5)

/disk2/user/hilpek/miniconda3/envs/cell2_env/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:167: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number
/disk2/user/hilpek/miniconda3/envs/cell2_env/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:283: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


In [20]:
# Spot × gene counts
pd.DataFrame(
    adata_vis.X.toarray() if hasattr(adata_vis.X, "toarray") else adata_vis.X.A,
    index=adata_vis.obs_names,
    columns=adata_vis.var_names
).T.to_csv("/disk2/user/hilpek/project_root/scRNAseq/input2/MappedDGEForR.csv")

# coordinates
coords = adata_vis.obsm['spatial']
coords_df = pd.DataFrame(coords, columns=["x", "y"], index=adata_vis.obs_names)
coords_df.to_csv("/disk2/user/hilpek/project_root/scRNAseq/input2/BeadLocationsForR.csv")

In [21]:
import pandas as pd

# load barcodes name
barcodes = adata_vis.obs_names

# Convert coords to DataFrame, assign barcodes as index
coords = pd.DataFrame(coords, index=barcodes, columns=["x", "y"])

# Convert indexes to R-compatible format (with dot and .1)
coords.index = coords.index.to_series().str.replace(r"^(V10F03)-", r"\1.", regex=True)
coords.index = coords.index.to_series().str.replace(r"-1$", ".1", regex=True)
